# AdaBoost
We want to create a model using AdaBoost for the 3 groupings: Control v. Parkinsonism, Idiopathic Parkinsonism vs Atypical Parkinsonism, and Multiple System Atrophy vs. Progressive Supranuclear Palsy

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import matthews_corrcoef, make_scorer, confusion_matrix

import ml_utils as mu

# groups to model
groupings = [
#     {'name': 'cvp', 'grouping':{0:0, 1:1, 2:1, 3:1}, 'description':'Control vs. All Parkinsons'},
#     {'name': 'iva', 'grouping':{1:1, 2:2, 3:2},'description':'Idiopathic vs. Atypical Parkinsons'},
    {'name': 'mvp', 'grouping':{2:2, 3:3},'description':'MSA vs. PSP'},
#     {'name': 'ipvm', 'grouping':{1:0, 2:1, 3:0},'description':'Idiopathic and PSP vs. MSA'},
#     {'name': 'imvp', 'grouping':{1:0, 2:0, 3:1},'description':'Idiopathic and MSA vs. PSP'},
]

# parameters to optimize on
params = {
            "classifier__n_estimators": range(1,401,200),
            "classifier__learning_rate": np.logspace(-4, -1, 30)
        }

# read in the data
data = pd.read_excel('data/all_data.xlsx').drop(['Subject'], axis=1)

# Things we could consider at this point are to remove subjects who are 
# younger than 55, and to remove controls who have a UPDRS > 5.

# for each grouping, split into test and holdout sets and train the model
for grouping in groupings:
    name = grouping['name']
    description = grouping['description']
    
    grouped_data = mu.group_classes(data, grouping['grouping'])
        
    y = grouped_data['GroupID']
    X = grouped_data.drop(['GroupID'], axis=1)
    
    X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size = 0.2, random_state = 43, stratify=y)

    pipeline = Pipeline([
        ("standardization", StandardScaler()),
        ('oversampler', RandomOverSampler()),
        ('kbest', SelectKBest(score_func=f_classif, k=7)),
        ("classifier", AdaBoostClassifier())
    ])
    
    clf = GridSearchCV(estimator=pipeline, param_grid=params, scoring=make_scorer(matthews_corrcoef), cv=2, n_jobs=-1, verbose=2)
    clf.fit(X_train, y_train)
    best_clf = clf.best_estimator_
    
    yh_pred = best_clf.predict(X_holdout)
    get_confusion_matrix(y_holdout, yh_pred, classes=description.split(' vs. '), title='Holdout: %s' %description)

Fitting 2 folds for each of 60 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    9.4s finished


In [4]:
import dill

f = open('experiments/FIRST_2_ADA_2018-07-31_20_55_31.obj','rb')
dill.load(f)

ModuleNotFoundError: No module named 'dill'

In [9]:
cf = confusion_matrix(y_holdout, yh_pred)
type(cf)

numpy.ndarray

In [2]:
import matplotlib.pyplot as plt
import itertools

def get_confusion_matrix(y, y_pred, classes, title="Confusion Matrix"):
    cnf_matrix = confusion_matrix(y, y_pred)
    plot_confusion_matrix(cnf_matrix, normalize=True, classes=classes, title=title)
    
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure()

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

### Stats to track:

* Model Name
* CV
* Datetime
* best parameters
* variant code
* description
* data file name
* validation data file name
* time_elapsed
*

For each cross_validation, holdout, validation:
* Sensitivity (Recall, True Positive Rate)
* Specificity (True Negative Rate)
* Precision  (Positive Predictive Value (PPV))
* Negative Predictive Value (NPV)
* Accuracy
* AUC
* MCC 

and standard deviation for each

### TODO:
* Setup db to track above stats on every run
* Run the full suite again @ 2,5,10,20 fold

Hopefully a model for each grouping where precision and recall > 0.75 (ideally > 0.80 )

### Variants to Try
* Don't run control vs. parkinsonism to save time
* Additional groupings for MSA vs. Idiopathic and PSP, PSP vs. Idiopathic and MSA
* PCA
* Feature Subset Selection
* Removing outliers
* Normalizing the input data
* Correcting for skewdness
* For MSA vs. PSP: MCP_FW (FA), SCP_FW, PUTAMEN_FW (FA), LOBULE_VI_FW,  LOBULE_V_FW
* ???? MCP/SCP ratio?